In [1]:
import h5py as h5
import hdf5storage
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import argparse
#from IPython.display import Image

In [2]:
#Read Picture and return it

def readThePicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)# | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    #old
    #imageio.plugins.freeimage.download()
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img) #returns picture

In [3]:
def tMO(file,name): #tonemapping the file
    try:
        if (name == 'reinhard'):
            tom = cv2.createTonemapReinhard()
        if (name == 'mantiuk'):
            tom = cv2.createTonemapMantiuk()
        if (name == 'drago'):
            tom = cv2.createTonemapDrago()
       # if (name == 'linear'):
        #    tom = cv2.createTonemap()
    except: 
        print('ToneMapping Error')
    ldr = tom.process(file)
    return ldr

In [4]:
def convert(img, target_type_min, target_type_max): # converts the input array to a target type with the bounderys given
    imin = img.min()
    imax = img.max()
    #print(imin)
    a = (target_type_max - target_type_min) / (imax - imin) # generates a factor a to multiply with the img
    b = target_type_max - a * imax
    new_img = (a * img + b)
    return new_img

In [5]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    try:
        #old
        #imageio.imwrite(outPath,picture,format=extention)#extention'')#save the Data (path with name and file extention,PictureMatrix,format)
        #new
        cv2.imwrite(outPath,picture)
        print(outPath+'<=== Writing')
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [6]:
def cutPatch(begX,begY,endX,endY,picyx):#cuts out a array of a given array
    try: 
        picyx = picyx[begY:endY,begX:endX] #format y,start:End | x,start:End
    except:
        print('FormatMaking Failed')
    return picyx #returns a small part of the pic file

In [7]:
def Randtone_map():
        #a random tonemapping is returned
        rand = random.SystemRandom()
        tmNumber = round((rand.randint(0, 20)/10)) # generates a random tonempaiing nuber 
        try:
            if (tmNumber<=0):
                return 'reinhard' #retruns the name of the tonemapper
            if (tmNumber==1):
                return 'mantiuk'
            if (tmNumber>=2):
                return 'drago'
           # if (tmNumber>=3):
            #    return 'linear'
        except:
            print('there was an tmo Error')
#ToDo Output in CSV to later analize

In [8]:
def totalpatchespossible(path,amountOfPictures,extention,px,py,tokonvPic): #calculates the amount of total possible patches of the path you picked 
    arraysize = 0 #zero the output
    amountOfPictures = amountOfPictures - 1
    tokonvPic= (amountOfPictures-(tokonvPic))+1 #generates the stop argument 
    while (amountOfPictures) >= tokonvPic:
        try:
            helping = os.listdir(path)[amountOfPictures].split('.')[1] #reading the File extention
            if ( helping == extention): #only counts files with the same extention
                he = (path+str(os.listdir(path)[amountOfPictures])) #reading path to picture
                print(he) #prints the name of the picture what is currently been read
                readPic = readThePicture(he)
                arraysize = arraysize + (int(readPic.shape[1]/px)*int(readPic.shape[0]/py))# calculate the whole size an cut away the rest even when 0.9
        except:
            print('fail count all patches')
        amountOfPictures = amountOfPictures - 1
    print('All patches will be: '+str(arraysize))
    return arraysize         

In [9]:
def patchesxy(inputpic,px,py): #calculates how often the array can be devided by px in x and py in y
    arraysize = []
    try:
        y = int(inputpic.shape[0]/py)# calculates the number of patches in the Y-axses cuts the picture i
        x = int(inputpic.shape[1]/px)
        arraysize = (x,y)
    except:
        print('fail calc x and y')
    return arraysize
        

In [10]:
def resizePic(inputpic,factor): #reszizing the inputpic picture keeping the information but scaling it down
    y = int((inputpic.shape[0])/factor) #multiply the Factor
    x = int((inputpic.shape[1])/factor) #multiply the Factor
    pic = cv2.resize(inputpic,(x,y)) 
    return pic

In [11]:
def RGBtoYUV(img): #changeing the img picture from RGB- to YUV-Color space
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return pictureYUV

In [12]:
def YUVtoRGB(img):#changeing the img picture from YUV- to RGB-Color space
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    return pictureRGB

In [13]:
#TO Finish
def inputargs():#todo finish
    parser = argparse.ArgumentParser()
    parser.add_argument('--foo', help='foo help')
    args = parser.parse_args()
    

In [29]:
#---- input section
#TO DO add parser
        
start_time = time.time() #start the timeing of the Prgramm
path = ''

#data extention of the input data
extention = input('What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]') or 'hdr'
print(extention)
#Where are the rawData at?
path = input('Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]') or './hdrInput/'
if not os.path.exists(path):
        os.mkdir(path)
print(path)
inhalte = (os.listdir(path)) #list of all files in path
amountOfPictures = 0

amountOfPictures = sum(1 for f in os.listdir(path) if f.endswith('.'+extention)) #summ all data ending with right extention
print('There are: '+str(amountOfPictures)+' '+extention+' Pictures in the folder')
#User can choose how many pictures should be cut in patches
tokonvPic = int(input('how many Pictures do you want to cut into patches? default 2: ') or '2')
print(str(tokonvPic)+' pictures will be cut into patches')
#scale factor for the low resolution is inputed
factor = int(input('Scale factor for Ldr LR [default:2]: ') or "2")
#asks for the px size of the high resolution pictures
xaxis = int(input('X Patch size HDR HR [default:320px]: ') or "320")
print(xaxis)
yaxis = int(input('Y Patch size HDR HR [default:320px]: ') or "320")
print(yaxis)
#user can choose if the pacht-pictures should be in YU-V or RGB
youWantYUV = input('Do you want to convert to yuv default: no') or 'no'
#user can coose in wich folder the .mat file is stored
savein = input('Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] ') or 'm'      
print(savein)
unit_varSdr = (np.float32)
unit_varHdr = (np.float32)
testing = input('Is the dataset for testing purposes? default: no') or 'no'
if (savein == 'm' or savein == 'mp'):
    #user can choose the name for the .mat file
    matName = input('Input Mat name default: data ') or 'data'
    matPath = input('Input Mat directory path: ./matOut/') or './matOut/'
    if not os.path.exists(matPath):
            os.mkdir(matPath)
    jsi = input('Is it for the JSI-GAN converion from float32 to uint8/16? default: no') or 'no'
    if (jsi != 'no'):
        unit_varSdr = (np.uint8)
        print('SDR .mat file will be uint8')
        unit_varHdr = (np.uint16)
        print('HDR .mat file will be uint16')
if (savein == 'p' or savein == 'mp'): #if user wants to output pates in picters he can choose where
    outPathsdr = input('spezify the output path of sdr pictures [default: ./sdrOut/] ') or './sdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathsdr):
            os.mkdir(outPathsdr)
    outPathhdr = input('spezify the output path of sdr pictures [default: ./hdrOut/] ') or './hdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathhdr):
        os.mkdir(outPathhdr)
    #TO DO if files should all have the same name or original Filename  

What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]
hdr
Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]
./hdrInput/
There are: 367 hdr Pictures in the folder
how many Pictures do you want to cut into patches? default 2: 
2 pictures will be cut into patches
Scale factor for Ldr LR [default:2]: 
X Patch size HDR HR [default:320px]: 
320
Y Patch size HDR HR [default:320px]: 
320
Do you want to convert to yuv default: no
Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] 
m
Is the dataset for testing purposes? default: no
Input Mat name default: data 
Input Mat directory path: ./matOut/
Is it for the JSI-GAN converion from float32 to uint8/16? default: noy
SDR .mat file will be uint8
HDR .mat file will be uint16


In [30]:
### write pic to .mat and/or .hdr/.png

#Just for general information Data Structure JSI-Gan
###['SDR_data'][79][79][2][39839] dtype=uint8 Strukture .mat Data
###['HDR_data'][79][79][2][39839] dtype=uint16 Strukture .mat Data
    

#---- programm section
allpatches = totalpatchespossible(path,amountOfPictures,extention,xaxis,yaxis,tokonvPic)  #calc all output patches
#print(str(allpatches)+'all Patches')

hdrarray = np.zeros((xaxis,yaxis,3,allpatches))#create empty np array of the size of allpatches
hdrarray = hdrarray.astype(unit_varHdr) #changes the type of np array to uint16
xldr = int(xaxis/factor) #calculates the samler array axes x
yldr = int(yaxis/factor)#calculates the samler array axes y
sdrarray = np.zeros((xldr,yldr,3,allpatches)) # creates the np array for the LR SDR array with new axes
sdrarray = sdrarray.astype(unit_varSdr)#changes the type of np array to uint8
#Arrays are defined in [amountOfallPatchesPossible,x,y,RGB]

print('Start processing...')
tokonvPic= (amountOfPictures-tokonvPic)# the amount of pictures cut into pachtes is calculated
allpatchesSave = allpatches #all pachtes are saved
amountOfPictures = amountOfPictures - 1 # needs to count down because while counts to more than in amonut of Pictures
while (amountOfPictures >= tokonvPic): #filling Array with pachtes from high to low, beginning with the hightes Number
    currentFile = os.listdir(path)[amountOfPictures] #currentFile holds the name of the current position file 
    try:
        if (currentFile.split('.')[1] == str(extention)): #checks if file is ending as wanted
            he = (path+str(currentFile))#gives the path and name with extention of the to process file
            print('processing the picture: '+he) #prints it to the user
            originalPicture = readThePicture(he) #reads the currentpicture and saves it
            
            print(originalPicture.max())
            begX = 1 #startingnumber in wich the cutting starts in X-axes
            endX = xaxis # sets current start pixel position at wich the cutting starts in X
            pxy=patchesxy(originalPicture,xaxis,yaxis) # gives back the length of the current picture (numx,numy) e.g. (3,2)
            px= pxy[0] #saves the max x pos in px
            py= pxy[1] #saves the max y pos in py
            
            while (px >= begX): # cut until you are at the beginning of the picture X position
                begY = 1 #startingnumber in wich the cutting starts in Y-axes
                endY = yaxis # sets current start pixel position at wich the cutting starts in Y
                x = (begX*xaxis)-xaxis #start patch position in x
    
                while (py>=begY): # cut until you are at the beginning of the picture Y position
                    ### reading the picture and cutting section
                    y = (begY*yaxis)-yaxis #start patch position in y
                    patch = cutPatch(x,y,endX,endY,originalPicture) #make the patch and return it to the patch (floart64) array
                    tmo = Randtone_map() # returns a random name for the tonemapping Operator to use
                    
                    patchLR = resizePic(patch,factor) #resize picture from Patch with the factor
                    tmoed = tMO(patchLR,tmo) #tonemap the resized patch with the before chosen Tone Mapping Operator
                    #print(tmoed)
                    patch_lrtm = tmoed
                    ####Color YUV Section
                    if (youWantYUV != 'no'): #if change to yuv
                        patch_lrtm = RGBtoYUV(tmoed) # changes the color space of lr sdr array
                        patch = RGBtoYUV(patch) # changes the color space of hr hdr array
                    ###Picture section
                    ### Pictures will be saved as 8bit .png with the picture beeing between 0- 255 uint8
                    ### and 16 bit .hdr with the picture beeing between 0- 2^16 float16
                    #print('HDR MAX')
                    if (savein == 'p' or savein == 'mp'): #save as picture if chosen
                        buildFilename = ((currentFile.split('.')[0])+'_'+str(allpatches-1))#builds output name 
                        # TODO Add a Input for the wanted out_format
                        if(testing != 'no'):
                            spaceIndi = 'u','v','y' #orders the Name to the right place
                            savePic((patch_lrtm[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'png',outPathsdr)#saves final singel color channel Picture
                            savePic((patch_lrtm[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'png',outPathsdr)#saves final singel color channel Picture
                            savePic((patch_lrtm[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'png',outPathsdr)#saves final singel color channel Picture
                            savePic((patch[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'hdr',outPathhdr)#saves final singel color channel Picture
                            savePic((patch[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'hdr',outPathhdr)#saves final singel color channel Picture
                            savePic((patch[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'hdr',outPathhdr)#saves final singel color channel Picture
                            ###To DO make 16 bit PNG
                        if(testing == 'no'):
                            savePic(patch,buildFilename,'hdr',outPathhdr)#change 'hdr' here for different HDR-picture save
                            savePic(patch_lrtm,buildFilename,'png',outPathsdr)#chnage 'png' here for different LDR-picture save 
                    #### uint Section
                    #print('ABC_1')
                    #print(patch_lrtm)
                    #patch_lr = patch_lrtm/((2 ** 8)-1) # normalize between 0,1 in float 32
                    #print('patchMax')
                    #print(patch.max())
                    #patch = patch/((2 ** 16)-1) #convert(patch,0,1) #normalize between 0,1 in float 32
                    if(unit_varHdr == np.uint16):
                        patch_lr = patch_lrtm * ((2 ** 8)-1) 
                        patch = patch * ((2 ** 10)-1) #HDR is changed so it fitts the unit16 OR UInt10?? format and can be compared
                        #print('HDR')
                        #print(patch.max())
                        #print(patch.min())
                        #print('LDR')
                        #print(patch_lr.max())
                        #print(patch_lr.min())
                        patch_lr = patch_lr.astype(unit_varSdr) #changes the data type in according to JSI Gan spec to uint8 for lr sdr
                        patch = patch.astype(unit_varHdr) #changes the data type in according to JSI Gan spec to uint16 for hr hdr
                    
                    ###writing array section 
                    p = (allpatches-1) #calcualte current patch position
                    try:
                        #print(patch_lr.shape)
                        sdrarray[:,:,:,p] = patch_lr # try Write the tmoed Patch to sdrarray at current patch position
                    except:
                        print('Error at SDR- Array Writing :..(')
                    try:
                        #print(patch.shape)
                        hdrarray[:,:,:,p] = patch # try Write the hr hdr Patch to hdrarray at current patch position
                    except:
                        print('Error at HDR- Array Writing :..(')
                        
                    begY = begY + 1 #Counts up from 1 to all possible cuts of the current picture in Y
                    endY = endY + yaxis #Counts up the position from endY in yaxis steps
                    allpatches = allpatches - 1 #Counts down all patches of all pictures to 0
                    print('Patch === '+str(allpatches)+' ==> Done')
                begX = begX + 1 #Counts up from 1 to all possible cuts of the current picture in X
                endX = endX + xaxis # Counts up the position from endX in xaxis steps
    except:
        print('Error with data maybe not an .hdr file continuing...')  
    amountOfPictures = amountOfPictures - 1 #counts down current picture pos
if (savein == 'mp' or savein == 'm' ): #only makes a Matlap File if wanted
    try:
        matLabel = 'HDR_data'
        if(testing != 'no'):
            matLabel = 'HDR_YUV'
            print('testdata')
        # Write TO HDR.Mat File
        h5.get_config().default_file_mode = 'a' #write enable
        matfilehdrdataHDR = {} # make a dictionary to store the MAT data in
        print('HDR Matlab file will have the format')
        print(hdrarray.shape)
        matfilehdrdataHDR[u''+matLabel] = hdrarray #save hdr array in that dictonary
        print('Writing HDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilehdrdataHDR, '.', matPath+'HDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the HDR .mat file')

        #####Writing SDR .mat 
        #Switches the first with the last array Field 
        matfilesdrdatasdr = {} # make a dictionary to store the MAT data in
        matLabel = 'SDR_data'
        if(testing != 'no'):
            matLabel = 'SDR_YUV'
            print('testdata')
        print('SDR Matlab file will have the format')
        print(sdrarray.shape)
        matfilesdrdatasdr[u''+matLabel] = sdrarray #save sdr array in that dictonary
        print('Writing SDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilesdrdatasdr, '.', matPath+'SDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the SDR .mat file')
    except:
        print('error at writing matlab file sorry :(')

print(str((time.time() - start_time)/60)+' Minutes') #outputs the time in minutes
print('------------------------- Done --------------------')

./hdrInput/blouberg_sunrise_1_16k.hdr
./hdrInput/old_depot_16k.hdr
All patches will be: 2550
Start processing...
processing the picture: ./hdrInput/blouberg_sunrise_1_16k.hdr
688.0
HDR
723.29297
307.69922
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2549 ==> Done
HDR
759.2578
335.67188
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2548 ==> Done
HDR
835.1836
375.6328
LDR
255.00003
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2547 ==> Done
HDR
907.1133
399.60938
LDR
255.00003
-1.811884e-12
(160, 160, 3)
(320, 320, 3)
Patch === 2546 ==> Done
HDR
1019.0039
447.5625
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2545 ==> Done
HDR
1358.6719
507.5039
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2544 ==> Done
HDR
1510.5234
687.3281
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2543 ==> Done
HDR
1814.2266
823.1953
LDR
255.0
1.811884e-12
(160, 160, 3)
(320, 320, 3)
Patch === 2542 ==> Done
HDR
4507.5938
1150.875
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2541 

HDR
1023.0
427.58203
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2445 ==> Done
HDR
1206.8203
487.52344
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2444 ==> Done
HDR
2061.9844
647.3672
LDR
254.99998
9.05942e-13
(160, 160, 3)
(320, 320, 3)
Patch === 2443 ==> Done
HDR
2749.3125
847.1719
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2442 ==> Done
HDR
3868.2188
1174.8516
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2441 ==> Done
HDR
4187.9062
1126.8984
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2440 ==> Done
HDR
12084.1875
911.1094
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2439 ==> Done
HDR
31201.5
177.82617
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2438 ==> Done
HDR
603.41016
32.967773
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2437 ==> Done
HDR
523.4883
73.927734
LDR
254.99998
3.623768e-12
(160, 160, 3)
(320, 320, 3)
Patch === 2436 ==> Done
HDR
483.52734
241.76367
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch ===

Patch === 2346 ==> Done
HDR
927.09375
383.625
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2345 ==> Done
HDR
1086.9375
495.51562
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2344 ==> Done
HDR
1550.4844
571.4414
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2343 ==> Done
HDR
2006.0391
735.28125
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2342 ==> Done
HDR
1934.1094
791.22656
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2341 ==> Done
HDR
1870.1719
855.16406
LDR
255.00003
3.623768e-12
(160, 160, 3)
(320, 320, 3)
Patch === 2340 ==> Done
HDR
2653.4062
699.3164
LDR
254.99998
3.623768e-12
(160, 160, 3)
(320, 320, 3)
Patch === 2339 ==> Done
HDR
2157.8906
23.47705
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2338 ==> Done
HDR
453.55664
21.978516
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2337 ==> Done
HDR
411.59766
36.71411
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2336 ==> Done
HDR
431.57812
136.86621
LDR
255.0
0.0
(160, 160, 3)


291.71484
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2247 ==> Done
HDR
787.23047
335.67188
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2246 ==> Done
HDR
835.1836
379.6289
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2245 ==> Done
HDR
891.1289
431.57812
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2244 ==> Done
HDR
955.0664
439.5703
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2243 ==> Done
HDR
1046.9766
531.48047
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2242 ==> Done
HDR
1046.9766
595.41797
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2241 ==> Done
HDR
819.1992
559.4531
LDR
255.00003
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2240 ==> Done
HDR
1254.7734
491.51953
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2239 ==> Done
HDR
735.28125
191.8125
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2238 ==> Done
HDR
703.3125
15.484863
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2237 ==> Done
HDR
427.58203
45.

HDR
667.34766
271.73438
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2148 ==> Done
HDR
703.3125
271.73438
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2147 ==> Done
HDR
751.2656
323.6836
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2146 ==> Done
HDR
799.21875
359.64844
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2145 ==> Done
HDR
871.14844
387.6211
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2144 ==> Done
HDR
899.1211
455.5547
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2143 ==> Done
HDR
919.10156
487.52344
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2142 ==> Done
HDR
887.1328
547.46484
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2141 ==> Done
HDR
771.2461
495.51562
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2140 ==> Done
HDR
683.33203
459.55078
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2139 ==> Done
HDR
587.4258
415.59375
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 2138 ==> Done
HDR
523.4883
150.85

...

Patch === 474 ==> Done
HDR
371.63672
0.842926
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 473 ==> Done
HDR
279.72656
0.7492676
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 472 ==> Done
HDR
226.77832
1.6234131
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 471 ==> Done
HDR
224.78027
8.491699
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 470 ==> Done
HDR
387.6211
0.62438965
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 469 ==> Done
HDR
10230.0
9.240967
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 468 ==> Done
HDR
547.46484
4.2458496
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 467 ==> Done
HDR
3868.2188
3.371704
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 466 ==> Done
HDR
21738.75
13.361938
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 465 ==> Done
HDR
24040.5
13.361938
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 464 ==> Done
HDR
27237.375
19.980469
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 463 ==> Done
HDR
577

Patch === 377 ==> Done
HDR
405.60352
20.47998
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 376 ==> Done
HDR
371.63672
23.976562
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 375 ==> Done
HDR
283.72266
11.988281
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 374 ==> Done
HDR
18541.875
11.48877
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 373 ==> Done
HDR
389.61914
0.62438965
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 372 ==> Done
HDR
259.7461
10.489746
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 371 ==> Done
HDR
273.73242
2.9970703
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 370 ==> Done
HDR
237.76758
2.3726807
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 369 ==> Done
HDR
431.57812
1.4985352
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 368 ==> Done
HDR
12531.75
4.7453613
LDR
255.0
5.6621374e-14
(160, 160, 3)
(320, 320, 3)
Patch === 367 ==> Done
HDR
11828.4375
4.6204834
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch

254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 280 ==> Done
HDR
711.3047
9.428284
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 279 ==> Done
HDR
583.4297
14.985352
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 278 ==> Done
HDR
771.2461
17.358032
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 277 ==> Done
HDR
353.6543
17.358032
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 276 ==> Done
HDR
371.63672
16.359009
LDR
254.99998
2.264855e-13
(160, 160, 3)
(320, 320, 3)
Patch === 275 ==> Done
HDR
279.72656
18.981445
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 274 ==> Done
HDR
22506.0
5.9941406
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 273 ==> Done
HDR
333.67383
6.993164
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 272 ==> Done
HDR
283.72266
1.6234131
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 271 ==> Done
HDR
273.73242
0.43707275
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 270 ==> Done
HDR
305.70117
0.99902344
LDR
25

HDR
433.57617
18.73169
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 181 ==> Done
HDR
511.5
16.85852
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 180 ==> Done
HDR
939.08203
19.980469
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 179 ==> Done
HDR
599.41406
13.736572
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 178 ==> Done
HDR
999.02344
12.612671
LDR
255.0
-1.1324275e-13
(160, 160, 3)
(320, 320, 3)
Patch === 177 ==> Done
HDR
319.6875
25.97461
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 176 ==> Done
HDR
499.51172
25.225342
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 175 ==> Done
HDR
283.72266
14.985352
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 174 ==> Done
HDR
329.67773
0.62438965
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 173 ==> Done
HDR
351.65625
0.062438965
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 172 ==> Done
HDR
377.63086
0.24975586
LDR
255.0
-2.8310687e-14
(160, 160, 3)
(320, 320, 3)
Patch === 171 

HDR
919.10156
16.23413
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 81 ==> Done
HDR
3900.1875
19.480957
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 80 ==> Done
HDR
30434.25
16.483887
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 79 ==> Done
HDR
9015.1875
8.366821
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 78 ==> Done
HDR
727.28906
8.491699
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 77 ==> Done
HDR
387.6211
27.972656
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 76 ==> Done
HDR
261.74414
29.471191
LDR
254.99998
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 75 ==> Done
HDR
323.6836
10.489746
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 74 ==> Done
HDR
281.7246
17.982422
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 73 ==> Done
HDR
331.67578
16.983398
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 72 ==> Done
HDR
399.60938
1.748291
LDR
255.0
0.0
(160, 160, 3)
(320, 320, 3)
Patch === 71 ==> Done
HDR
387.6211
2.3726807
LDR
255.0
0.0

In [41]:
#python3 main.py --phase train --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --epoch 5 --batch_size 4

In [31]:
### gives the amount of nuberst larger than 0 in percent
#np.count_nonzero(ab)
#(np.count_nonzero(sdrarray)/sdrarray.size)*100

99.98023845996732

In [32]:
#np.count_nonzero(ab)
#(np.count_nonzero(hdrarray)/hdrarray.size)*100

99.99268407883987

In [21]:
#python3 train.py --batch_size 100 -d ./hdrData -s ./pathToSaceChepoints 

In [20]:
#python3 main.py --phase test_mat --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --test_data_path_LR_SDR /home/nico/programm/MdataPicPrep/matOut/SDR_data.mat --test_data_path_HR_HDR /home/nico/programm/MdataPicPrep/matOut/HDR_data.mat --batch_size 4